In [9]:
from tinydb import TinyDB, Query
from datetime import datetime

Record = Query()

db_take_profit = TinyDB('trailing_take_profit.json')
db_stop_loss = TinyDB('trailing_stop_loss.json')
db_logs = TinyDB('logs.json')


In [10]:
STOP_LOSS = 0.004       # i.e. default = 0.4%
TAKE_PROFIT = 0.01      # i.e. default = 1%
ENABLE_STOP_LOSS = True
ENABLE_TAKE_PROFIT = True


In [11]:
def updateOrCreate(coin_record,db):
    found = len(db.search(Record.symbol == coin_record.get('symbol')))
    # print(f"found {found}")
    if found > 0:
        db.update(coin_record, Record.symbol == coin_record.get('symbol'))
    else:
        db.insert(coin_record)
        

def getCoinRecord(symbol,db):
    arr = db.search(Record.symbol == symbol)
#     print(f'arr {arr}')
    if(len(arr) <= 0):
        return {}
    return arr[0]

def removeCoinRecord(symbol,db):
    try:
        db.remove(Record.symbol == symbol)
        return True
    except Exception as err:
        print(f"err {err}")
        return False
    

In [21]:
symbol = "BTCUSDT"
current_price = 12000
buy_price = 10000
take_profit = 11000
stop_loss = 9000
tralling_take_profit = 0.05
tralling_stop = 1 - stop_loss/buy_price
stop_order = 0


coin_record = {"symbol": symbol,"buy_price": buy_price, "stop_loss": stop_loss, 'tralling_stop': tralling_stop, "stop_order": stop_order , "createdAt": datetime.now().strftime("%d/%m/%Y %H:%M:%S")}
# coin_record = {"symbol": symbol,"buy_price": buy_price, "take_profit": take_profit, "tralling_take_profit": tralling_take_profit, "stop_order": stop_order , "createdAt": datetime.now().strftime("%d/%m/%Y %H:%M:%S")}
# coin_record = {"symbol": symbol,"stop_order": stop_order , "createdAt": datetime.now().strftime("%d/%m/%Y %H:%M:%S")}
print(coin_record)
updateOrCreate(coin_record, db_stop_loss)

{'symbol': 'BTCUSDT', 'buy_price': 10000, 'stop_loss': 9000, 'tralling_stop': 0.09999999999999998, 'stop_order': 0, 'createdAt': '27/08/2021 07:26:10'}


In [22]:
# def do_stop_order(symbol, )
coin_record = getCoinRecord(symbol, db_stop_loss)
# coin_record = getCoinRecord(symbol, db_take_profit)
coin_record

{'symbol': 'BTCUSDT',
 'buy_price': 10000,
 'stop_loss': 9000,
 'stop_order': 0,
 'createdAt': '27/08/2021 07:26:10',
 'tralling_stop': 0.09999999999999998}

In [23]:
def trailing_take_profit(symbol, current_price):
    coin_record = getCoinRecord(symbol, db_take_profit)
    if(coin_record.get('symbol')):
        if(current_price < coin_record.get('take_profit')):
            return False
        else:
            
            current_stop_order = coin_record.get('stop_order')
            if(current_price <= current_stop_order):
                return True
            
            new_stop_order = current_price*(1-coin_record.get('tralling_take_profit'))
            coin_record = {"symbol": symbol,"stop_order": max(current_stop_order, new_stop_order) , "createdAt": datetime.now().strftime("%d/%m/%Y %H:%M:%S")}
            print(coin_record)
            updateOrCreate(coin_record, db_take_profit)
        
    return False

def trailing_stop_loss(symbol, current_price):
    coin_record = getCoinRecord(symbol, db_stop_loss)
    if(coin_record.get('symbol')):
        if(current_price <= coin_record.get('stop_loss')):
            return True
        else:
            
            current_stop_order = coin_record.get('stop_order')
            if(current_price <= current_stop_order):
                return True
            
            new_stop_order = current_price*(1-coin_record.get('tralling_stop'))
            coin_record = {"symbol": symbol,"stop_order": max(current_stop_order, new_stop_order) , "createdAt": datetime.now().strftime("%d/%m/%Y %H:%M:%S")}
            print(coin_record)
            updateOrCreate(coin_record, db_stop_loss)
        
    return False

In [25]:
symbol = "BTCUSDT"
# current_price = 12000
buy_price = 1000
# take_profit = 11000
stop_loss = 900
# tralling_take_profit = 0.05
tralling_stop = 1 - stop_loss/buy_price
stop_order = 0


coin_record = {"symbol": symbol,"buy_price": buy_price, "stop_loss": stop_loss, "tralling_stop": tralling_stop, "stop_order": stop_order , "createdAt": datetime.now().strftime("%d/%m/%Y %H:%M:%S")}
# coin_record = {"symbol": symbol,"buy_price": buy_price, "take_profit": take_profit, "tralling_take_profit": tralling_take_profit, "stop_order": stop_order , "createdAt": datetime.now().strftime("%d/%m/%Y %H:%M:%S")}
# coin_record = {"symbol": symbol,"stop_order": stop_order , "createdAt": datetime.now().strftime("%d/%m/%Y %H:%M:%S")}
print(coin_record)
updateOrCreate(coin_record, db_stop_loss)

{'symbol': 'BTCUSDT', 'buy_price': 1000, 'stop_loss': 900, 'tralling_stop': 0.09999999999999998, 'stop_order': 0, 'createdAt': '27/08/2021 07:28:05'}


In [26]:
# buy btc 10000
# ENABLE_TAKE_PROFIT
enable_take_loss = trailing_stop_loss("BTCUSDT", 1000)
print(f"enable_take_loss {enable_take_loss}")
# print(f"enable_take_profit {enable_take_profit}")
coin_record = getCoinRecord("BTCUSDT", db_stop_loss)
coin_record

{'symbol': 'BTCUSDT', 'stop_order': 900.0, 'createdAt': '27/08/2021 07:28:17'}
enable_take_loss False


{'symbol': 'BTCUSDT',
 'buy_price': 1000,
 'stop_loss': 900,
 'stop_order': 900.0,
 'createdAt': '27/08/2021 07:28:17',
 'tralling_stop': 0.09999999999999998}

In [27]:
# symbol = "BTCUSDT"
# current_price = 12000
# buy_price = 10000
# take_profit = 11000
# tralling_take_profit = 0.05
# stop_order = 0


# coin_record = {"symbol": symbol,"buy_price": buy_price, "take_profit": take_profit, "tralling_take_profit": tralling_take_profit, "stop_order": stop_order , "createdAt": datetime.now().strftime("%d/%m/%Y %H:%M:%S")}
# # coin_record = {"symbol": symbol,"stop_order": stop_order , "createdAt": datetime.now().strftime("%d/%m/%Y %H:%M:%S")}
# print(coin_record)
# updateOrCreate(coin_record)
# buy btc 10000
# ENABLE_TAKE_PROFIT
enable_take_loss = trailing_stop_loss("BTCUSDT", 1010)
print(f"enable_take_loss {enable_take_loss}")
# print(f"enable_take_profit {enable_take_profit}")
coin_record = getCoinRecord("BTCUSDT", db_stop_loss)
coin_record


{'symbol': 'BTCUSDT', 'stop_order': 909.0, 'createdAt': '27/08/2021 07:30:21'}
enable_take_loss False


{'symbol': 'BTCUSDT',
 'buy_price': 1000,
 'stop_loss': 900,
 'stop_order': 909.0,
 'createdAt': '27/08/2021 07:30:21',
 'tralling_stop': 0.09999999999999998}

In [28]:
# enable_take_profit = trailing_take_profit("BTCUSDT", 10000)
# print(f"enable_take_profit {enable_take_profit}")
# coin_record = getCoinRecord("BTCUSDT")
# coin_record

enable_take_loss = trailing_stop_loss("BTCUSDT", 1250)
print(f"enable_take_loss {enable_take_loss}")
# print(f"enable_take_profit {enable_take_profit}")
coin_record = getCoinRecord("BTCUSDT", db_stop_loss)
coin_record

{'symbol': 'BTCUSDT', 'stop_order': 1125.0, 'createdAt': '27/08/2021 07:31:49'}
enable_take_loss False


{'symbol': 'BTCUSDT',
 'buy_price': 1000,
 'stop_loss': 900,
 'stop_order': 1125.0,
 'createdAt': '27/08/2021 07:31:49',
 'tralling_stop': 0.09999999999999998}

In [29]:
# enable_take_profit = trailing_take_profit("BTCUSDT", 10500)
# print(f"enable_take_profit {enable_take_profit}")
# coin_record = getCoinRecord("BTCUSDT")
# coin_record


enable_take_loss = trailing_stop_loss("BTCUSDT", 1200)
print(f"enable_take_loss {enable_take_loss}")
# print(f"enable_take_profit {enable_take_profit}")
coin_record = getCoinRecord("BTCUSDT", db_stop_loss)
coin_record

{'symbol': 'BTCUSDT', 'stop_order': 1125.0, 'createdAt': '27/08/2021 07:32:58'}
enable_take_loss False


{'symbol': 'BTCUSDT',
 'buy_price': 1000,
 'stop_loss': 900,
 'stop_order': 1125.0,
 'createdAt': '27/08/2021 07:32:58',
 'tralling_stop': 0.09999999999999998}

In [30]:
# enable_take_profit = trailing_take_profit("BTCUSDT", 12000)
# print(f"enable_take_profit {enable_take_profit}")
# coin_record = getCoinRecord("BTCUSDT")
# coin_record


enable_take_loss = trailing_stop_loss("BTCUSDT", 1125)
print(f"enable_take_loss {enable_take_loss}")
# print(f"enable_take_profit {enable_take_profit}")
coin_record = getCoinRecord("BTCUSDT", db_stop_loss)
coin_record

enable_take_loss True


{'symbol': 'BTCUSDT',
 'buy_price': 1000,
 'stop_loss': 900,
 'stop_order': 1125.0,
 'createdAt': '27/08/2021 07:32:58',
 'tralling_stop': 0.09999999999999998}

In [85]:
enable_take_profit = trailing_take_profit("BTCUSDT", 11000)
print(f"enable_take_profit {enable_take_profit}")
coin_record = getCoinRecord("BTCUSDT")
coin_record

enable_take_profit True


{'symbol': 'BTCUSDT',
 'buy_price': 10000,
 'take_profit': 11000,
 'tralling_take_profit': 0.05,
 'stop_order': 11400.0,
 'createdAt': '25/08/2021 12:01:36'}

In [87]:
removeCoinRecord("BTCUSDT")
coin_record = getCoinRecord("BTCUSDT")
coin_record

{}